In [1]:
import csv
import sys
import argparse
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
SQL_TEMPLATE = """

WITH plan_id AS (
    SELECT pp.pricing_plan_id
    FROM resource_tracker_pricing_plans pp
    WHERE pp.pricing_plan_key = '{pricing_plan_key}'
      AND pp.product_name = '{product_name}'
    LIMIT 1
),
new_licensed_item AS (
    INSERT INTO licensed_items
        (licensed_resource_type, 
         pricing_plan_id, 
         product_name, 
         created, 
         modified,
         display_name, 
         key, 
         version, 
         is_hidden_on_market
        )
    SELECT
        'VIP_MODEL',
        plan_id.pricing_plan_id,
        '{product_name}',
        now(),
        now(),
        '{display_name}',
        '{license_key}',
        '{license_version}',
        {is_hidden_in_market}
    FROM plan_id
    RETURNING licensed_item_id
),
matching_resources AS (
    SELECT 
      licensed_resource_id
    FROM licensed_resources
    WHERE licensed_resource_name IN ({id_list})
),
insert_relationships AS (
    INSERT INTO licensed_item_to_resource
        (licensed_item_id, licensed_resource_id, created, modified)
    SELECT
        new_licensed_item.licensed_item_id,
        matching_resources.licensed_resource_id,
        now(),
        now()
    FROM new_licensed_item
    CROSS JOIN matching_resources
    RETURNING licensed_item_id, licensed_resource_id
)

SELECT * FROM insert_relationships;

-------------------------------------------------------------------------------
"""

In [5]:
INPUT_PRODUCT = "s4lacad"
INPUT_PRODUCT = "s4l"

In [6]:
INPUT_PRODUCT = "s4lacad"

In [7]:
EXCEL_PP_MAPPING = {
    f"{INPUT_PRODUCT}-vip-vip-v4": "ViP V4",
    f"{INPUT_PRODUCT}-vip-vip-v3": "ViP V3",
    f"{INPUT_PRODUCT}-vip-morphed-v3": "Morphed V3 (*)",
    f"{INPUT_PRODUCT}-vip-vip-v1": "ViP V1",
    f"{INPUT_PRODUCT}-vip-vizoo-v1": "ViZoo V1",
    f"{INPUT_PRODUCT}-vip-vizoo-v1-pig": "ViZoo V1 – Pig",
    f"{INPUT_PRODUCT}-vip-vizoo-v3-monkey": "ViZoo V3 – Monkey",
    f"{INPUT_PRODUCT}-vip-vizoo-v4-neurorat": "ViZoo V4 – NeuroRat",
    f"{INPUT_PRODUCT}-vip-head-bundle": "Head bundle", 
    f"{INPUT_PRODUCT}-vip-free-vip": "Free VIP", 
    f"{INPUT_PRODUCT}-vip-mida": "MIDA", 
}

all_sql_statements = []

for key, value in EXCEL_PP_MAPPING.items():
    INPUT_PRICING_PLAN_KEY = key
    
    #############################################
    
    csv_file_path = "VIP Model licensing Sim4Life.web - mapping_table - 21022025.csv"
    df = pd.read_csv(csv_file_path)
    df_filtered = df[df["PRICING"] == EXCEL_PP_MAPPING[INPUT_PRICING_PLAN_KEY]]  # <-- CHANGE PRICING PLAN!
    grouped = df_filtered.groupby(["key (OSPARC)", "version (OSPARC)"])
    assert len(grouped) > 0, "TIP: probably wrong PRICING PLAN filter"
    
    #############################################
    sql_statements = []
    
    for (k, v), group_df in grouped:
        print(f"Group for key={k}, version={v}:")
        # Each `group_df` contains rows that share the same key and version
        
        id_list = group_df['licensed_resource_name'].tolist()
    
        assert len(id_list) == len(set(id_list))
        _licensed_item_display_name = set(group_df['display_name (OSPARC)'].tolist())
        assert len(_licensed_item_display_name) == 1
        _licensed_item_pricing = set(group_df['PRICING'].tolist())
        assert len(_licensed_item_pricing) == 1
        _licensed_item_is_hidden_on_market = set(group_df['Shown on Market'].isnull().tolist())
        assert len(_licensed_item_is_hidden_on_market) == 1
    
        print(f"Display name: {_licensed_item_display_name}, is hidden on market {_licensed_item_is_hidden_on_market}")
        
        print(f"All IDs ({len(group_df)}) in this group:", id_list)
        print("-----------------------------------")
    
        _display_name_str = _licensed_item_display_name.pop()
        _is_hidden_on_market_str = _licensed_item_is_hidden_on_market.pop()
        _id_list_str = ",".join(f"\'{i}\'" for i in id_list)
        
        sql_block = SQL_TEMPLATE.format(
                pricing_plan_key=INPUT_PRICING_PLAN_KEY,
                product_name=INPUT_PRODUCT,
                # pricing_plan_id=PRICING_PLAN_ID,
                display_name=_display_name_str,
                license_key=k,
                license_version=v,
                is_hidden_in_market=_is_hidden_on_market_str,
                id_list=_id_list_str
            )
        sql_statements.append(sql_block)
        all_sql_statements.append(sql_block)
    
    
    #############################################
    
    output_file = f"import-{key}.sql"
    
    # Write all SQL blocks to the output file (or you could print them)
    # with open(output_file, 'w', encoding='utf-8') as out_f:
    #     for block in sql_statements:
    #         out_f.write(block)

# Write all SQL blocks to the output file (or you could print them)
with open(f"import-all.sql", 'w', encoding='utf-8') as out_f:
    out_f.write("BEGIN;")
    for block in all_sql_statements:
        out_f.write(block)
    out_f.write("COMMIT;")

Group for key=CMODEL_JEDUK_POSABLE, version=4.0.0:
Display name: {'Jeduk Posable V4.0'}, is hidden on market {False}
All IDs (1) in this group: ['HumanWholeBody/119']
-----------------------------------
Group for key=CMODEL_BILLIE, version=3.0.0:
Display name: {'Billie V3.0 (Not shown)'}, is hidden on market {True}
All IDs (1) in this group: ['HumanWholeBody/72']
-----------------------------------
Group for key=CMODEL_BILLIE_POSABLE, version=3.1.0:
Display name: {'Billie Posable V3.1'}, is hidden on market {False}
All IDs (2) in this group: ['HumanWholeBody/51', 'HumanWholeBody/155']
-----------------------------------
Group for key=CMODEL_BILLIE_POSABLE (NOT_SHOWN), version=3.0.0:
Display name: {'Billie V3.0 (Not shown)'}, is hidden on market {True}
All IDs (1) in this group: ['HumanWholeBody/50']
-----------------------------------
Group for key=CMODEL_DIZZY, version=3.0.0:
Display name: {'Dizzy V3.0 (Not shown)'}, is hidden on market {True}
All IDs (1) in this group: ['HumanWholeBo

In [54]:
# LICENSED TEXT

In [9]:
SQL_TEMPLATE_LICENSE_TEXT = """BEGIN;

UPDATE licensed_resources
SET licensed_resource_data = jsonb_set(licensed_resource_data, '{{terms_of_use_url}}', '"{url}"')
WHERE licensed_resource_name IN ({licensed_resource_name});

COMMIT;

-------------------------------------------------------------------------------
"""

In [10]:
#############################################

csv_file_path = "VIP Model licensing Sim4Life.web - mapping_table - 21022025.csv"
df = pd.read_csv(csv_file_path)
df = df.reset_index()  # make sure indexes pair with number of rows

sql_statements = []
for index, row in df.iterrows():
    if pd.isna(row['license_url']):
        continue
    print(row['licensed_resource_name'], row['license_url'])
    
    sql_block = SQL_TEMPLATE_LICENSE_TEXT.format(
            url=row['license_url'],
            licensed_resource_name=f"\'{row['licensed_resource_name']}\'"
        )
    sql_statements.append(sql_block)


#############################################

output_file = "test.sql"

# Write all SQL blocks to the output file (or you could print them)
with open(output_file, 'w', encoding='utf-8') as out_f:
    for block in sql_statements:
        out_f.write(block)

HumanWholeBody/6 https://raw.githubusercontent.com/ITISFoundation/licenses/refs/heads/main/models/User%20License%20Virtual%20Population%20Models%20v4.x%2C%20v3.x%20and%20v1.x.md
HumanWholeBody/7 https://raw.githubusercontent.com/ITISFoundation/licenses/refs/heads/main/models/User%20License%20Virtual%20Population%20Models%20v4.x%2C%20v3.x%20and%20v1.x.md
HumanWholeBody/8 https://raw.githubusercontent.com/ITISFoundation/licenses/refs/heads/main/models/User%20License%20Virtual%20Population%20Models%20v4.x%2C%20v3.x%20and%20v1.x.md
HumanWholeBody/9 https://raw.githubusercontent.com/ITISFoundation/licenses/refs/heads/main/models/User%20License%20Virtual%20Population%20Models%20v4.x%2C%20v3.x%20and%20v1.x.md
HumanWholeBody/10 https://raw.githubusercontent.com/ITISFoundation/licenses/refs/heads/main/models/User%20License%20Virtual%20Population%20Models%20v4.x%2C%20v3.x%20and%20v1.x.md
HumanWholeBody/11 https://raw.githubusercontent.com/ITISFoundation/licenses/refs/heads/main/models/User%20Lic

In [79]:
# LICENSED RESOURCE PRIORITY

In [11]:
SQL_TEMPLATE_LICENSE_RESOURCE_PRIORITY = """BEGIN;

UPDATE licensed_resources
SET priority = {priority}
WHERE licensed_resource_name IN ({licensed_resource_name});

COMMIT;

-------------------------------------------------------------------------------
"""

In [12]:
#############################################

csv_file_path = "VIP Model licensing Sim4Life.web - mapping_table - 21022025.csv"
df = pd.read_csv(csv_file_path)
df = df.reset_index()  # make sure indexes pair with number of rows

sql_statements = []
for index, row in df.iterrows():
    if pd.isna(row['priority']):
        continue
    print(row['licensed_resource_name'], int(row['priority']))
    
    sql_block = SQL_TEMPLATE_LICENSE_RESOURCE_PRIORITY.format(
            priority=int(row['priority']),
            licensed_resource_name=f"\'{row['licensed_resource_name']}\'"
        )
    sql_statements.append(sql_block)


#############################################

output_file = "test.sql"

# Write all SQL blocks to the output file (or you could print them)
with open(output_file, 'w', encoding='utf-8') as out_f:
    for block in sql_statements:
        out_f.write(block)

HumanWholeBody/18 0
HumanWholeBody/19 1
HumanWholeBody/20 2
HumanWholeBody/143 1
HumanWholeBody/144 1
HumanWholeBody/146 1
HumanWholeBody/147 1
HumanWholeBody/148 1
HumanWholeBody/149 1
HumanWholeBody/150 1
HumanWholeBody/151 1
HumanWholeBody/153 1
HumanWholeBody/154 1
HumanWholeBody/155 1
